## Load Dataset from csv

In [1]:
from pyspark.sql import DataFrame
#Load a csv file and convert to DataFrame
df = spark.read.option("header","true").csv("data/nepal_train.csv")
df = df.select(df.tweet_text,df.label.cast("double").alias("label"))
df = df.dropna()
#df.write.csv("data/mycsv.csv")
df.createOrReplaceTempView("tweets")
all_tweets = spark.sql("SELECT tweet_text, label FROM tweets")

In [2]:
only_tweets = spark.sql("SELECT tweet_text FROM tweets")

# Feature extraction

## Tokenize

In [3]:
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType

tokenizer = Tokenizer(inputCol="tweet_text", outputCol="words")
#regexTokenizer = RegexTokenizer(inputCol="tweet_text", outputCol="words", pattern="\\W")
# alternatively, pattern="\\w+", gaps(False)

countTokens = udf(lambda words: len(words), IntegerType())
tokenized = tokenizer.transform(all_tweets)
#tok = tokenized.select("label", "words")

tokenized = tokenized.withColumn("tokens", countTokens(col("words")))
#tok = tokenized.select("words","tokens","label")    

## Stop words remover

In [4]:
from pyspark.ml.feature import StopWordsRemover

remover = StopWordsRemover(inputCol="words", outputCol="filtered")
filtered = remover.transform(tokenized)
#filtered = filtered.select("filtered","tokens","label")
#filtered.show()

## NGram

In [5]:
from pyspark.ml.feature import NGram

ngram = NGram(n=3, inputCol="filtered", outputCol="ngrams")
ngrams = ngram.transform(filtered)
#ngramDataFrame = ngramDataFrame.select("ngrams")
#ngramDataFrame.show()

## HashingTF and IDF

In [6]:
from pyspark.ml import linalg as ml_linalg

def as_mllib(v):
    if isinstance(v, ml_linalg.SparseVector):
        return MLLibVectors.sparse(v.size, v.indices, v.values)
    elif isinstance(v, ml_linalg.DenseVector):
        return MLLibVectors.dense(v.toArray())
    else:
        raise TypeError("Unsupported type: {0}".format(type(v)))

In [7]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.ml.linalg import Vector as MLVector, Vectors as MLVectors
from pyspark.mllib.linalg import Vector as MLLibVector, Vectors as MLLibVectors
from pyspark.mllib.regression import LabeledPoint
from pyspark.ml.feature import VectorAssembler

hashingTF = HashingTF(inputCol="ngrams", outputCol="rawFeatures", numFeatures=50000)
featurizedData = hashingTF.transform(ngrams)
# alternatively, CountVectorizer can also be used to get term frequency vectors
idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
tfidfData = idfModel.transform(featurizedData)
assembler = VectorAssembler(
    inputCols=["features", "tokens"],
    outputCol="union_features")
assembledData = assembler.transform(tfidfData)
pairs = assembledData.select("label","union_features").rdd
data = pairs.map(lambda x: LabeledPoint(x[0], as_mllib(x[1])))

In [8]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[tokenizer, remover, ngram, hashingTF, idf, assembler])
#pipeline.save("target/tmp/pythonHashingTF")

In [11]:
pipeModel = pipeline.fit(all_tweets)
#pipeModel.save("target/tmp/pythonHashingTFModel_new")

In [12]:
pipeModel.transform(all_tweets)

IllegalArgumentException: u'Field "tokens" does not exist.'

In [9]:
from pyspark.ml import Pipeline
pipe = Pipeline.load("target/tmp/pythonHashingTF")

In [3]:
from pyspark.ml import PipelineModel
pipeModel1 = PipelineModel.load("target/tmp/pythonHashingTFModel_new")

In [14]:
tf_df = pipeModel.transform(only_tweets)
tf_df.show()

IllegalArgumentException: u'Field "tokens" does not exist.'

# Train a model

## Linear SVM

In [11]:
# Split the data into train and test
splits = data.randomSplit([0.6, 0.4], 1234)
train = splits[0]
test = splits[1]

In [12]:
from pyspark.mllib.classification import SVMWithSGD, SVMModel

# Build the model
model = SVMWithSGD.train(train, iterations=100)

# Evaluating the model on training data
labelsAndPreds = train.map(lambda p: (p.label, model.predict(p.features)))
trainErr = labelsAndPreds.filter(lambda (v, p): v != p).count() / float(train.count())
print("Training Error = " + str(trainErr))
# Save and load model
#model.save(sc, "target/tmp/pythonSVMWithSGDModel")
#sameModel = SVMModel.load(sc, "target/tmp/pythonSVMWithSGDModel")

Training Error = 0.0777221294751


In [ ]:
model.save(sc, "target/tmp/pythonSVMWithSGDModel_new")